# Welcome to Week 4: State-Space \& Graph Traversal

In this week's programming exercise, you will be working on several problems with different state-spaces and how to search for answers in these state-spaces. Throughout the exercise, you will be extending the classes by completing code stubs in their respective cells. You do not need to copy the code, it is enough to work in the cell under each exercise. Note that there are separate cells provided where you can (and should) test your code. During the exercises, you will (through customMagics) obtain a Python file (.py) which you should run against a set of unittests. Please avoid writing any unnecessary code in cells containing the `%%execwritefile` command. Doing this could alter the file `.py` and make it syntactically incorrect or interfere with the unittests. To prevent this stick to the following rules:'
 - ***Do not remove cells that start with ``%%execwritefile`` and do not remove that line.***
 - If a cell contains a `%%execwritefile` command at the top and a class definition you need to complete the given methods and adding helper methods is allowed, but do **not** add new functions or Python script to the cells (like global variables).
 - If a cell contains a `%%execwritefile` command at the top and **not** a class definition you must complete the given functions and you are free to add helper functions, new classes, and Python script that contains for example global variables. Note, that the use of global variables is almost always wrong except for a few use cases such as RNG for the numpy random generator methods.
 - If a cell does **not** contain a `%%execwritefile` command you can plot things, print variables, and write test cases. Here, you are free to do whatever you want.
 - If a cell does **not** contain a `%%execwritefile` command it should not contain functional code that is needed to run other functions or classes. The reason is that it is not copied to the `.py`. So, it can not be used during the unittesting.

You do not need to look at the `customMagic.py` nor do more than glimpse at the test file, your exercise is contained in this workbook unless specified differently in this notebook's instructions. 

***Hint: Jupyter Notebooks saves variables between runs. If you get unexpected results try restarting the kernel, this deletes any saved variables.*** 

Please fill in your student name down below

In [74]:
# FILL IN YOU STUDENT NUMBER
student = 4003748

# Set this to false if you want the default screen width.
WIDE_SCREEN = True

In [75]:
from custommagics import CustomMagics
import timeit
import matplotlib.pyplot as plt

if WIDE_SCREEN:
    import notebook
    from IPython.display import display, HTML

    if int(notebook.__version__.split(".")[0]) >= 7:    
        display(HTML(
            '<style>'
                '.jp-Notebook { padding-left: 1% !important; padding-right: 1% !important; width:100% !important; } '
            '</style>'
        ))
    else:
        display(HTML("<style>.container { width:98% !important; }</style>"))

get_ipython().register_magics(CustomMagics)



In [76]:
%%execwritefile exercise4_{student}_notebook.py 0 

# DO NOT CHANGE THIS CELL.
# THESE ARE THE ONLY IMPORTS YOU ARE ALLOWED TO USE:

import numpy as np
import copy
from collections import defaultdict, deque

RNG = np.random.default_rng()

exercise4_4003748_notebook.py is backup to exercise4_4003748_notebook_backup.py
Overwriting exercise4_4003748_notebook.py


In [77]:
plt.matplotlib.rcParams['figure.figsize'] = [8, 3]

# Classes as Functions \& The `__call__` Method

This week, we will use classes a bit different than we have seen so far. Therefore, please read this explanation if you are not familiar with the idea that an object (of a class) can be used as a function. Similar to last week, you do not have to make your own callable classes because the framework is already given but it helps you understand how it works.

## Callable Object

The term callable object refers to objects that are callable aka where you can give it some input in round brackets and they return an answer. So far all of you have used these kinds of objects namely when making functions. The name of functions is technically also a callable object. Below you can see a function called `some_function` which makes `some_function` an object which can be called as follows: `some_function(10)`. Thus `some_function` is an object and when you put round brackets `()` behind it you execute the code that you wrote in the `def`. 

```python
def some_function(variable1):
    print(variable1)
```

Some of you also might have seen lambda functions which also create callable objects but they are not directly stored in a variable. Therefore they are called a nameless function. However, you could give still store it in a variable which makes that variable a callable object. Below, we used a lambda function to create the same functionality as the `def` function above. Note, this is not how you should use lambda functions, but it illustrates what a callable object is.

```python
some_function = lambda variable1: print(variable1)
```

## Creating Callable Objects with Classes

While functions are very useful, they can quickly explode in complexity. To solve this you can create helper functions, however, if all your helper functions and main function use the same variables it becomes very messy if you add a lot of arguments for each (helper) function. You might be tempted to solve this with global variables but this is really bad practice because if a file contains multiple algorithms and helper functions you do not know which global variable belongs to which algorithm. A more structured way of solving this is using classes, where you can add as much helper methods in the class as you want and you can use the object attributes to store variables that all (helper) methods need. This brings us to the magic method `__call__`, if you implement the `__call__` method in a class the objects of the class become callable and when you call them the `__call__` method is executed. To demonstrate this we will make a class that creates objects with the same functionality as the function above. In the example below `some_function` can now be used as a "normal" function where you call it as follows: `some_function(10)`.

```python
class SpecialFunction():
    def __call__(self, variable1):
        print(variable1)

some_function = SpecialFunction()  # create the callable object
```

Now, you can add extra (helper) methods to your class to make the code more structured and readable and use object attributes to store variables across multiple methods. Often, these kind of classes do not have an `__init__` method and the attributes are initialized in `__call__` depending on the given arguments of `__call__`. In exercises [2](#2.0-State-Space-for-Permutations) and [3.3.1](#3.3.1-Breadth-First-Search-Framework) you can find frameworks for depth-first and breadth-first search that use these kind of classes.

# 1.0 Implicit State-Space Graph for Alternating Disks

In the theory questions, you designed an algorithm to solve the alternating disks problem (question 5). Here, you will program your solution. However, before you start think about the state-space and if you actively track it in your algorithm. By actively tracking, we mean that you can easily go back one step and make a different choice. Don't worry if your approach looks something like this: Go through the list, if you encounter a light disk to the left of a dark disk swap them, and keep going through the list until all the light disks are to the left of the dark disks. This is a perfect solution for the problem, however, for more difficult problems that have multiple solutions (and we want all of them) we might need a more structured approach. This will be tackled in problem [2.0 State-Space of Permutations](#2.0-State-Space-of-Permutations).
   


In [78]:
%%execwritefile exercise4_{student}_notebook.py 10 -a -s

def alternating_disks(n):
    """
    This function solves the alternating disks problem for a row with size 2*n containing n light disks and n dark disks.
    The function returns an array with n ones (light disks) and n zeros (dark disks).
    Use the function swap to change the position of two disks.
    
    :param n: Number of light or dark disks
    :type n: int
    :return: The ordered list (light is 1 and dark is 0)
    :rtpye: np.ndarray[(2*n), int]
    """
    disks = np.array(list(zip(np.ones(n), np.zeros(n)))).flatten()
    i = 0
    while 1 in disks[n:] and i < 2 * n - 2:
        if disks[i] != disks[i + 1] and disks[i] == 0:
            swap(disks, i)
        i += 1
        # we might need several traversals through the list
        if i == 2 * n - 2:
            i = 0
    return disks
def swap(disks, i):
    """
    This function swaps the disks i and i+1 in the array disks.
    This is a helper function for alternating_disks.

    :param disks: Array containing the light and dark disks
    :type disks: np.ndarray[(2*n), int]
    :param i: Position of the disk that needs to be swapped
    :type i: int
    """
    disks[i], disks[i + 1] = disks[i + 1], disks[i]

Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your code for the function `alternating_disks`.

In [79]:
# write here your code to test alternating_disks
alternating_disks(150)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# 2.0 State-Space for Permutations

Above, we implemented `alternating_disks` using a function that only uses actions that we know lead to the correct solution. So, the state-space only contains states and actions that lead to the solution. However, in each state, the possible correct actions are hard to predetermine as they depend on which state-space you are currently in. In a relatively simple algorithm, this is not hard to implement in a single function where we just quickly search for it. This is basically what the step "Go through the list and if you encounter a light disk to the left of a dark disk swap them" does. However, making a single function, for a recursive search algorithm that needs to return every possible solution path (a series of actions that lead to a correct solution), is relatively hard. Therefore, we will introduce a framework that can be used for any state-space problem. This framework can then either only use valid state-spaces, e.g., as we saw in the lecture in the wolf, goat, and cabbage problem, or can use any state-space defined by all actions. For example, for the wolf, goat, and cabbage problem, this would mean that the first action is not only limited to taking the goat but can also be taking the wolf or cabbage.

We will do a similar exercise by making permutations of a list. First, we will use a state-space that only leads to correct solutions then we will use a more general state-space filter out wrong solutions. To do all this we will make use of the framework explained below.

***Before, you start programming think about how to make permutations on paper and think about a pseudo algorithm.***

In [80]:
class DepthFirstSearchFramework():
    def __call__(self, problem):
        """
        This method is used to initialize the problem and solve it.

        Hint: __call__ is the magic method used when using round brackets, 
        .i.e, it makes the object callable. For example:
            s = StateSpace()
            s(problem)  # This calls the __call__ method of the object s.
        """
        ...
        results = self.step(...)
        ...
        return ...
    
    def step(self, args):
        """
        Step is the recursive part of the depth-first search.
        Each step contains of several substeps:
         - Check if you are in the base case and act upon it.
         - Go through all possible actions you can take in this state-space and for each action do the following:
           - Apply the action, this can be changing the state-space or adding something the a list.
           - Go to the next step, this makes it depth-first search as you don't explore all options 
             fist but do one action and then go to the next. In next_step, you can also generate 
             the next possible actions if needed.
             When going back up the tree to return the value you can also do some updating.
             For example, in permutations you can all permutations of the next step and you have to 
             loop through them to generate all permutations including the chosen action.
           - Do clean up, for example if you delete something from a list to do the step 
             then you need to reverse it to be able to do another action.
             Another, (expensive) option instead of cleanup would be copying statespace,
             so you do not need to reverse the action as you still have the original statespace.
            
        Note, that in this template we made a method for next step, but you can do the same for
        applying the action, clean up, or the base case. Often, you do this to make your code more
        structured, i.e., more readable.
        """
        # base case
        if ...:
            return ...

        # chose one action and go to the next step
        for action in actions:
            # Apply the action
            ...

            self.next_step()
            ...
            # keep going if you want all possible solutions or return here if one solution is enough.

            # Clean up
            ...

    def next_step(self):
        """
        This method helps us determine the next actions given the current state-space.

        If each state-space this method does not do much, but it still can adjust a counter 
        to know which recursive depth we are if this is needed. 
        However, if each state-space has its own set of actions for example in the wolf, goat, cabbage problem
        You determine the next actions in this method.
        """
        # generate next action
        ...

        # call step
        self.step()

    def is_correct(self):
        """
        If you depth-first search can find state-spaces that are incorrect you need some method to check if the state-space is correct.

        This method can be called at three points in your algorithm:
         - At the end (in __call__), you go through all answer and filter them.
         - If you reach the base-case, this is often done if you are only interested in one result.
         - When you take an action, to check if the partial answer is correct. We comeback to this in lecture 5.
        """
        if ...:
            return True
        return False    

In [81]:
%%execwritefile exercise4_{student}_notebook.py 20 -a -s

class Permutations():
    def __call__(self, list_):
        """
        This method gives all the permutations of the list.
        This is done by generating them first using step and
        then filtering them using `self.is_correct`.

        :param list_: The list containing all the elements.
        :type list_: list[Objects]
        :return: A list with all possible permutations.
        :rtype: list[list[Objects]]
        """
        # Nothing needs to be initialized or filter. So just call self.step
        return self.step(list_)
        
    def step(self, list_):
        """
        This method adds one value to the new permutated list and
        and calls next_step to generate a new set of actions.

        :param list_: The list containing all the possible elements (that are valid actions)
        :type list_: list
        :return: A list containing all the permutations, from the current space-state.
        :type: list[list[Objects]]
        """
        # base case
        if len(list_) == 0:
            return [[]]
        
        permutations = []
        for index in range(len(list_)):
            for action in self.next_step(list_, index):
                action.append(list_[index])
                permutations.append(action)
        
        return permutations

        



    def next_step(self, list_, index):
        """
        This method generates the actions that are possible for the next step and calls step.
        These actions consist of the elements of the list that are not yet in the new permutated list.

        :param list_: The list containing all the possible elements (that are valid actions)
                      from the previous state-space.
        :type list_: list
        :param index: The index of the element that is used as action in the previous step.
        :type index: int
        :return: This method returns what self.step returns
        :type: list[list[Objects]]
        """
        # generate next action
        new_list = list_
        new_list = new_list[:index] + new_list[index + 1:]
        return self.step(new_list)

Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your permutation algorithm.

In [82]:
# Type your testing code here
Permutations()([1,2,3])

[[3, 2, 1], [2, 3, 1], [3, 1, 2], [1, 3, 2], [2, 1, 3], [1, 2, 3]]

## 2.1 Permutations with Replacement

Above, we implemented a permutation algorithm that only finds correct state-spaces but we need to do extra work to generate the actions. Another approach would be to use another state-space, where we define the state-space as a list where each index can be any element of the original list independent of what the other indices contain. In other words, we use the same state-sapce that you would use to make permutations with replacement.

This makes `next_step` a lot easier, however, now we need `is_correct` to filter out incorrect solutions aka we have to search through the whole state-space which final states are correct and which are not.

**Note, in this exercise we assume that all elements in the list are unique!**

***Before, you start programming think about how to make permutations on paper if the indices are independent and think about a pseudo algorithm.***

In [83]:
%%execwritefile exercise4_{student}_notebook.py 25 -a -s

class PermutationsWithReplacement():
    def __call__(self, list_):
        """
        This method gives all the permutations of the list.
        This is done by generating them first using step and
        then filtering them using `self.is_correct`.
        
        :param list_: The list containing all the unique elements.
        :type list_: list
        :return: A list with all possible permutations.
        :rtype: list[list[Objects]]
        """
        # all actions are the same, so it is helpful to make an object attribute.
        self.list = list_
        # filter out all incorrect state-spaces and return all permutations of self.list
        return [x for x in self.step(0) if self.is_correct(x)]

    def step(self, i):
        """
        This method adds one value to the new permutated list and
        and calls next_step to generate a new set of actions.
        
        :param i: A counter how many elements are added to the new permutation.
        :type i: int
        :return: A list containing all the permutations, from the current space-state.
        :type: list[list[Objects]]
        """
        # add all combinations of elemnets to the new list
        # base case
        if i == len(self.list):
            return [[]]
        
        
        permutations = []
        for index in range(len(self.list)):
            # call the next step to generate available actions
            for action in self.next_step(i):
                action.append(self.list[index])
                permutations.append(action)
                
        return permutations
            
            

    def next_step(self, i):
        """
        This method generates the actions that are possible for the next step and calls step.
        These actions consist of all elements of the original list.

        :param i: An counter how many elements are added to the new permutation.
        :type i: int
        :return: This method returns what self.step returns
        :type: list[list[Objects]]
        """
        return self.step(i + 1)
    
    def is_correct(self, permutation):
        """
        This method returns if the state-space is correct aka if it is a permutation.

        :param permutation: A possible permutation of self.list
        :type permutation: list[Objects]
        :return: Return if the permutation variable is or is not a permutation.
        :rtype: boolean
        """
        # check if the list contains unique elements
        if len(permutation) == len(set(permutation)):
            return True
        return False
    


Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your permutation algorithm using the class `PermutationsWithReplacement`. Also, check if it is the same as the previous class Permutations.

In [84]:
# Type your testing code here
PermutationsWithReplacement()([1,2,3])

[[3, 2, 1], [2, 3, 1], [3, 1, 2], [1, 3, 2], [2, 1, 3], [1, 2, 3]]

# 3.0 Different State-Spaces

For a lot of problems, there is no unambiguous state-space or even an unambiguous datastructure that fits the problem. This is also shown above. There are many trade-offs when it comes down to choosing a correct state-space for your problem and this is often connected to what kind of algorithm you use. In this exercise, we will focus on two different state-spaces but also two different datastructures to see what the difference can be.

## 3.1 Solving The Manhattan Problem Using Arrays

The problem we want to solve is to find every possible fastest route in a small part of Manhattan. As input, we have a simplified photograph of the area and we want to go from the top left corner of the map to the bottom right corner, see the image below.

![Manhattan.png](Manhattan.png)

With a bit of computer-vision this image can be transformed into the following array, where a 1 is a road and 0 is a house:
```python
road_grid = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
])
```

So, given the array above we can make a search algorithm that finds all the fastest routes. However, there are a few obstacles in designing such an algorithm. One obstacle is that it is easy to walk in circles or to find a suboptimal route. Therefore, in this problem we assume that going right and down is always fast and going up or left is always wrong (as in you create a loop or you are slower).

Given the framework above, design an algorithm that works for every possible Manhattan grid, given the assumptions above. This problem is inspired by the [Manhattan distance/geometry](https://en.wikipedia.org/wiki/Taxicab_geometry) (just for a bit of background you do not need it to make the algorithm. 

***Before you start coding, try to work out an example on paper and write down a pseudo algorithm. Also, think about what the state-space in this case is.***


In [85]:
%%execwritefile exercise4_{student}_notebook.py 30 -a -s

class ManhattanProblem():
    def __call__(self, road_grid):
        """
        This method gives all the fastest routes through this part of Manhattan.
        You always start top left and end bottom right.
        This is done by calling step which should return a list of routes, 
        where a route consists of a list of coordinates.

        :param road_grid: The array containing information where a house (zero) or a road (one) is.
        :type road_grid: np.ndarray[(Any, Any), int]
        :return: A list with all possible routes, where a route consists of a list of coordinates.
        :rtype: list[list[tuple[int]]]
        """
        self.grid = road_grid
        return self.next_step((0,0))  # We already are in the first state-space so we need to generate the next actions.
        
    def step(self, pos, actions):
        """
        This method does one step in the depth-first search in the Manhattan grid.
        One step consists of adding one coordinate (tuple) to the route and
        generating all possible routes from this coordinate in the grid,
        this is done recursively.

        :param pos: The current coordinate in the grid.
        :type pos: tuple[int]
        :param actions: List of possible next coordinates.
        :type actions: list[tuple[int]]
        :return: All possible route with this position as starting point.
        :rtype: list[list[tuple[int]]]
        """
        if pos == (len(self.grid) - 1, len(self.grid[0]) - 1):
            return [[pos]]
        
        paths = []
        for action in actions:
            for path in self.next_step(action):
                paths.append([pos] + path)
        
        return paths


    def next_step(self, pos):
        """
        Here, we check which actions we can take depending on the current position in the grid.
        Then, we call next step with the current position and next possible actions.

        :param pos: The current coordinate in the grid.
        :type pos: tuple[int]
        :return: This method returns what self.step returns
        :rtype: list[list[tuple[int]]]
        """
        pos_candidates = []
        grid = self.grid
        if pos[0] + 1 < len(grid) and grid[pos[0] + 1][pos[1]] == 1:
            pos_candidates.append((pos[0] + 1, pos[1]))
        if pos[1] + 1 < len(grid[0]) and grid[pos[0]][pos[1] + 1] == 1:
            pos_candidates.append((pos[0], pos[1] + 1))
        
        return self.step(pos, pos_candidates)

Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your algorithm. Make sure that you test various road grids and that your algorithm works for all of them. To give you a head start, a simple one-block road network is provided.

In [86]:
# Test your code here

# road_grid = np.array([
#     [1, 1, 1, 1, 1, 1],
#     [1, 0, 0, 0, 0, 1],
#     [1, 0, 0, 0, 0, 1],
#     [1, 0, 0, 0, 0, 1],
#     [1, 1, 1, 1, 1, 1],
# ])
road_grid = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
])
# print(road_grid[0][0])
for route in ManhattanProblem()(road_grid):
    print(route)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12)]
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (5, 12), (6, 12), (7, 12), (8, 12)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (8, 12)]
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (4, 6), (4, 7), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8),

## 3.2 The Manhattan Problem Using Graphs

In the previous solution for the Manhattan problem, we used a state-space that included all road positions. However, this uses a lot of resources not only in memory but also in speed. A lot of decisions in this state-space could be phrased in English as "just keep going". The graph representation of this state-space would look something like:

<img src="ManhattanGraph.png" alt="drawing" width="700"/>

If you look at the graph above, you could say that all nodes that are "just keep going" nodes, can be removed from the graph as they do not change the outcome of the routes. This would result in the graph below, which is much smaller and quicker to traverse. However, making this graph is not as straight forward and a separate algorithm is required to do it. This leads to the trade-off of how many times are you using the graph and is it worth it to transform it to the smaller graph representation. The important is that both are valid state-space representations.

<img src="ManhattanGraphSmall.png" alt="drawing" width="200"/>

Below, you can find a Node class that is used to build a graph representing the road network. Please, make sure you understand the attributes and how this class would make a whole graph. You do not have to build anything with the class but the Node class represents the graph. Of course, this problem could also be solved using an adjacency list. Here, we use a Node class to let you practice with different datastructures. You can also find an algorithm that transforms an array representation into a small graph representation. This is given to let you test both algorithms on the same array. You can ignore the code itself which is also NOT an example of how to write good code. 

In [87]:
%%execwritefile exercise4_{student}_notebook.py 32 -a -s

class Node():
    """
    This Node class forms a graph where each node contains directed edges to the other nodes.

    Attributes:
        :param info: The coordinate of the Node
        :type info: tuple[(2,), int]
        :param edges: A list of Nodes which are the directed edges from this Node.
        :type edges: list[Node]
    """
    def __init__(self, info):
        self.info = info
        self.edges = []

    def set_edges(self, edges):
        self.edges = edges

    def __repr__(self):
        # return f"Node{self.info}"  # you can choose which representation you like or make one yourself.
        return f"Node{self.info} -> {[e.info for e in self.edges]}"


Appending to exercise4_4003748_notebook.py


In [88]:
%%execwritefile exercise4_{student}_notebook.py 33 -a -s

def make_graph(grid):
    # if the grid is effectively one dimensional
    if 1 in grid.shape:
        start = Node((0,0))
        start.edges = [Node((grid.shape[0]-1, grid.shape[1]-1))]
        return start
    
    # add start and end nodes
    row_col_nodes = defaultdict(list)
    col_row_nodes = defaultdict(list)
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            row_adj = sum([row[j-1], row[j+1]] if j > 0 and j < grid.shape[1]-1 else [row[j+1] if j == 0 else row[j-1]])  # check if there is an adjacent road horizontally
            col_adj = sum([grid[i-1, j], grid[i+1, j]] if i > 0 and i < grid.shape[0]-1 else [grid[i+1,j] if i == 0 else grid[i-1,j]]) # check if there is an adjacent road vertically
            # junction if both vertically and horizontally there is a road and you are standing on a road. 
            if cell and row_adj and col_adj:
                row_col_nodes[i] += [j]
                col_row_nodes[j] += [i]

    # make nodes
    nodes = {}
    for row, cols in row_col_nodes.items():
        for col in cols:
            nodes[(row, col)] = Node((row,col))

    # connect nodes
    for (row, col), node in nodes.items():
        edges = []
        if col + 1 < grid.shape[1] and grid[row, col + 1]:  # go left and find next node
            edges.append(nodes[row, row_col_nodes[row][row_col_nodes[row].index(col)+1]])
        if row + 1 < grid.shape[0] and grid[row + 1, col]: # go down and find next node
            edges.append(nodes[col_row_nodes[col][col_row_nodes[col].index(row)+1], col])
        node.set_edges(edges)
            
    return nodes[(0,0)]

Appending to exercise4_4003748_notebook.py


### 3.2.1 Solve The Problem

In the exercise below, you will make an algorithm to traverse the smaller graph. You can use the function `make_graph` to test your algorithm.

In [89]:
%%execwritefile exercise4_{student}_notebook.py 35 -a -s

class ManhattanProblemDepth():
    def __call__(self, road_graph):
        """
        This method gives all the fastest routes through this part of Manhattan.
        You start with the first node `road_graph` and you end if you are at the end node.
        You can assume there are no dead ends in the graph.
        This is done by calling step which should return a list of routes, 
        where a route consists of a list of coordinates.

        :param road_graph: The start Node of the graph.
        :type road_graph: Node
        :return: A list with all possible routes, where a route consists of a list of coordinates.
        :rtype: list[list[tuple[int]]]
        """
        return self.step(road_graph)
        
    def step(self, node):
        """
        This method does one step in the depth-first search in the Manhattan grid.
        One step consists of adding one coordinate (tuple) to the route and
        generating all possible routes from this coordinate in the grid.

        :param node: A Node, that contains the current coordinate in the grid.
        :type node: Node
        :return: All possible routes with this position as starting point.
        :rtype: list[list[tuple[int]]]
        """
        if not node.edges:
            return [[node.info]]

        paths = []
        for edge in node.edges:
            for path in self.next_step(edge):
                paths.append([node.info] + path)

        return paths

    def next_step(self, node):
        """
        Becaues, we are traversing the state-space graph itself explicitly, 
        there is nothing to do in next_step, as the next actions are encoded by the edges.

        :param node: A Node, that contains the current coordinate in the grid.
        :type node: Node
        :return: This method returns what self.step returns
        :rtype: list[list[tuple[int]]]
        """
        return self.step(node)

Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your algorithm. Make sure that you test various road grids and that your algorithm works for all of them. To give you a head start, a simple one-block road network is provided. Also, check the difference between the two algorithms. Are the paths different? Does one generate more paths?

In [90]:
road_grid = np.array([
    [1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1],
])

road_graph = make_graph(road_grid)
print(ManhattanProblemDepth()(road_graph))
print(ManhattanProblem()(road_grid))

[[(0, 0), (0, 5), (4, 5)], [(0, 0), (4, 0), (4, 5)]]
[[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5)], [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 5), (2, 5), (3, 5), (4, 5)]]


## 3.3 Breadth-First Search on the Manhatten 

Now, that we have the road graph we can also apply a breadth-first search to it. 

### 3.3.1 Breadth-First Search Framework

Often breadth-first search is implemented using a while loop and a queue. In each iteration, you explore all possible paths for each state in the queue. So, in contrast to depth-first search, we do not need any recursion. However, the recursion did a lot of "remembering" for us. Now, we really need to store all the information ourselves. Therefore, dividing your code into several methods is still good practice. Below, you can find a framework to do this.

***Hint: deque (queue and stack) has special methods to make it function as a queue and stack compared to a normal list, see [documentation](https://docs.python.org/3/library/collections.html#collections.deque.append).***

In [91]:
class BreadthFirstSearchFramework():
    def __call__(self, problem):      
        """
        Here, you can initialize all variables to solve this problem.
        You always need a queue and history. 
        However, it might also be useful to have another variable to store the end result.

        The history can store only the nodes that you visited (thus a set) or store the nodes 
        that you visited and some information about them (thus a dictionary).
        This information can be how you got here or the current cost, etc.
        """ 
        self.queue = deque(...)  # propper datastructure, however, a list would work as well
        self.history = {...}
        self.store_result = ...

        # call the main loop
        self.main_loop(...)

        # return the answer(s)
        return self.store_result

    def main_loop(self, args):
        """
        This method "controls" the breadth-first search.
        Here, you loop through the queue and each iteration.
        you check for the base case (end node) and 
        take the steps that you can take from this node.
        """
        # keep going through the queue until ...
            # pop the first item
            
            # check base case
            
            # Loop through all actions and take these steps
            # for action in self.next_step():
            #     ...

    def base_case(self, args):
        """
        Here, you can check if you are in the base case or a final node.
        If so do the necessary steps. Depending on the problem you can either
        return if you found the base case or not return anything.
        This could help control if you to stop searching or not.
        """
        if ...:
            # Do base case action
            ...

        # Possibly return something
        return ...

    def step(self, args):
        """
        Here, you can write code to take one step.

        Often a step involves checking if you already visited the node and
        depending on the answer do something else. For example, often if you 
        already visited the node you do not add it again to the queue, however,
        you might want to do something with the history (if you store additional information in it).
        """
        if ...:
            # append node to queue
            ...
            
            # update history
            self.update_history(...)
        else:
            # update history (optional)
            self.update_history(...)

    def next_step(self, args):
        """
        Similar to depth-first search if it is not trivial what the next actions are
        it is good to make a separate method for them.
        """
        # Gather possible action given the current state
        ...

    def update_history(self, args):
        """
        If you have a more complex history then just the set of nodes you visited.
        It might help to have a separate method to handle the history.
        """
        # update or set the history
        ...
    

In [113]:
%%execwritefile exercise4_{student}_notebook.py 37 -a -s

class ManhattanProblemBreadth():   
    def __call__(self, road_graph):      
        """
        This method gives all the fastest routes through this part of Manhattan.
        You start with the first node `road_graph` and you end if you are at the end node.
        You can assume there are no dead ends in the graph.

        Hint: The history is already given as a dictionary with as keys all the nodes in the state-space graph and
        as values all possible routes that lead to this node.

        This class instance should at least contain the following attributes after being called:
            :param queue: A queue that contains all the nodes that need to be visited.
            :type queue: collections.deque
            :param history: A dictionary containing the nodes that are visited and all routes that lead to that node.
            :type history: dict[Node, set[tuple[tuple]]]

        :param road_graph: The start Node of the graph.
        :type road_graph: Node
        :return: A list with all possible routes, where a route consists of a list of coordinates.
        :rtype: list[list[tuple[int]]]
        """
        self.queue = deque([road_graph])  # propper datastructure, however, a list would work as well
        self.history = {road_graph: {(road_graph.info,)}}
        self.final_node = None
        self.main_loop()
        output_list = sorted(list(self.history[self.final_node]))
        # output_list = self.history[self.final_node]
        new_list = [list(path) for path in output_list]
        return new_list

    def main_loop(self):
        """
        This method contains the logic of the breadth-first search for the Manhattan problem.

        It does not have any inputs nor outputs. 
        Hint, use object attributes to store results.
        """
        while self.queue:
            current_node = self.queue.popleft()
            if self.base_case(current_node):
                 continue
            for sub_node in self.next_step(current_node):
                self.step(current_node, sub_node)

            

    def base_case(self, node):
        """
        This method checks if the current node is the base code, i.e., final node.

        :param node: The current node
        :type node: Node
        """

        if not node.edges:
            self.final_node = node
            return True
        
        return False
            
    def step(self, node, new_node):
        """
        One breadth-first search step.
        Here, you add new nodes to the queue and update the history.

        :param node: The current node
        :type node: Node
        :param new_node: The next node that can be visited from the current node
        :type new_node: Node        
        """
        if node not in self.queue:
            self.queue.append(new_node)
            self.update_history(node, new_node)


    def next_step(self, node):
        """
        This method returns the next possible actions.

        :param node: The current node
        :type node: Node
        :return: A list with possible next nodes that can be visited from the current node.
        :rtype: list[Node]  
        """
        return node.edges

    def update_history(self, node, new_node):
        """
        For more complex histories it is good to have a separate method to 
        set or update the history.
        
        :param node: The current node
        :type node: Node
        :param new_node: The next node that can be visited from the current node
        :type new_node: Node    
        """
        if new_node not in self.history.keys():
            self.history[new_node] = set()
        for path in self.history[node]: 
            self.history[new_node].add(path + (new_node.info,))
            # self.history[new_node]
    


Replace existing code exercise4_4003748_notebook.py


## Test your code

Below, you can test your algorithm. Make sure that you test various road grids and that your algorithm works for all of them. To give you a head start, a simple one-block road network is provided. Also, check the difference between the three algorithms. Are the paths different? Does one generate more paths?

In [114]:
road_grid = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
])

road_graph = make_graph(road_grid)
print(ManhattanProblemBreadth()(road_graph))
# for route in ManhattanProblemBreadth()(road_graph):
#     print(route)
# for route in ManhattanProblemDepth()(road_graph):
#     print(route)
# for route in ManhattanProblem()(road_grid):
#     print(route)

[[(0, 0), (0, 5), (0, 8), (0, 12), (4, 12), (8, 12)], [(0, 0), (0, 5), (0, 8), (4, 8), (4, 12), (8, 12)], [(0, 0), (0, 5), (0, 8), (4, 8), (8, 8), (8, 12)], [(0, 0), (0, 5), (4, 5), (4, 8), (4, 12), (8, 12)], [(0, 0), (0, 5), (4, 5), (4, 8), (8, 8), (8, 12)], [(0, 0), (0, 5), (4, 5), (8, 5), (8, 8), (8, 12)], [(0, 0), (4, 0), (4, 5), (4, 8), (4, 12), (8, 12)], [(0, 0), (4, 0), (4, 5), (4, 8), (8, 8), (8, 12)], [(0, 0), (4, 0), (4, 5), (8, 5), (8, 8), (8, 12)], [(0, 0), (4, 0), (8, 0), (8, 5), (8, 8), (8, 12)]]


# 4.0 Tower of Hanoi

## ***All exercises below are additional content for extra practice, they only count for a small percentage of the bonus (TestExpert). So, you can skip to section [5.0 Unittests](#5.0-UNITTESTS) if you do not want to do these exercises.***

In Introduction To Programming, we have seen an algorithm to solve the tower of Hanoi which does not use the state-spaces but generates a series of instructions how you can solve any tower as fast as possible, see [link](https://joshhug.github.io/LeidenITP/labs/lab10/#exercise-7-tower-of-hanoi-challenge-hard-or-very-hard) for an explanation how this fast approach works. However, this is not an easy algorithm to come up with. Often the fastest algorithm to solve a problem is unknown which means that we need to use search algorithms. Therefore, we will implement a depth-first search and breadth-first search to solve the Tower of Hanoi. To make it a bit easier we only consider actions that generate correct state-spaces and we are not interested in the fastest path to solve the problem. So, any path that leads to a correct solution is good. Below, you can see the state-space of the tower of Hanoi with 3 disks if you only allow valid actions.

<img src="hanoi.png" alt="drawing" width="600"/>

As you can see, this state-space has a lot of cycles in the solution graph. Finding a correct solution path in a cyclic state-space graph is much harder than in a state-space that is a tree or at least an acyclic graph. The reason is that you can keep taking actions that leads you in circles and you never reach an end node. One way to solve this problem is to remember which state you already visited and discard any path that visited a state more than once.

## 4.1 Tower of Hanoi Fast Algorithm (no search)

Let's first implement the Fast algorithm, such that we have a way to know for any problem size (number of disks) what the best solution path is, see [link](https://joshhug.github.io/LeidenITP/labs/lab10/#exercise-7-tower-of-hanoi-challenge-hard-or-very-hard) for explanation. Note, that this algorithms does not search through the tree because the first recursive step just assumes that it can find a solution to move a tower with size n-1. The state-spaces that are visited in the first recursive step can be visualized as seen below.

<img src="hanoi_fast.png" alt="drawing" width="600"/>


In [94]:
%%execwritefile exercise4_{student}_notebook.py 40 -a -s

class TowerOfHanoiFast():
    def __call__(self, n):
        """
        This method solves the tower of Hanoi according to the fast algorithm.
        The output should be a list containing all the moves to solve the problem.
        A move is a tuple with two integers containing from which rod to which rod.
        For example, move (0,2) is place the top disk of rod 0 onto rod 2.
        Also, the attribute towers should contain the end solution where all disks are 
        in the correct order on rod 2.

        This class instance should at least contain the following attributes after being called:
            :param tower: A tuple with the tree rods each rod is represented by a list 
                          with integers where the integer is the size of the disk.
            :type tower: tuple[list[int]]

        Hint 1: It might be easier to first generate the moves and then apply them to the towers.
        Hint 2: The argument "n" is not needed for the algorithm but can help. 
                It is however needed if you do hint 1.

        :param n: This is the height of the tower, i.e., the number of disks.
        :type n: int
        :return: A list containing all the moves to solve the tower of Hanoi with n disks.
        :rtype: list[tuple[int]]
        """
        self.towers = (list(range(n, 0, -1)), [], [])

        raise NotImplementedError("Please complete this method")

    def step(self, source, aux, dest, n=0):
        """
        This method can be used as the recursive part of the algorithm

        :param source: The rod that has the current (sub)tower that needs to be moved.
        :type source: int
        :param aux: The auxiliary rod that can be used to transfer all disks to the destination rod.
        :type aux: int
        :param dest: The destination rod where the (sub)tower needs to go.
        :type dest: int
        :param n: The height of the current tower that is moved, default to 0.
        :type n: int, optional
        """
        raise NotImplementedError("Please complete this method")

Appending to exercise4_4003748_notebook.py


## Test your code

Below, you can test your algorithm. Make sure that you test various numbers of disks and that your algorithm works for all of them. To give you a head start, the tower of Hanoi with 3 disks is provided.

In [95]:
moves = TowerOfHanoiFast()(3)
moves, len(moves)

NotImplementedError: Please complete this method

## 4.2 Tower of Hanoit Depth-First Search

Here, we will implement a depth-first search algorithm to traverse the state-space of any tower of Hanoi problem. There is a new problem when traversing the state-space of a tower of Hanoi problem because we can go in circles as the graph is cyclic. This can also be seen in the state-space images of the tower of Hanoi. One way of solving this problem is to add some kind of memory to the depth-first-search algorithm that prevents us from walking in circles. The easiest way to accomplish this is to have a new attribute that stores which states are already visited and when you come across a state you already visited you just go back and don't continue exploring. This problem is similar to the breadth-first search history problem.

The possible actions for each state-space are given in the class attribute `possible_action`. The path you will find with depth-first search given this order does not give the optimal solution. Is there an order of `possible_action` actions that leads to the optimal solution? Below, you can see the solution you should find when using the given order of possible actions.

<img src="hanoi_depth.png" alt="drawing" width="600"/>

***Before, you start programming think about a pseudo algorithm that could traverse the state-space of a tower of Hanoi with 3 disks as shown above.***

In [ ]:
%%execwritefile exercise4_{student}_notebook.py 44 -a -s

class TowerOfHanoiDepth():
    # All possible actions for any tower of Hanoi State-space
    possible_actions = [(0, 1), 
                        (0, 2), 
                        (1, 2), 
                        (2, 1), 
                        (1, 0), 
                        (2, 0)] 

    def __call__(self, n):
        """
        This method uses depth-first search to find a solution to solve the tower of Hanoi.
        The output should be a list containing all the moves to solve the problem.
        A move is a tuple with two integers containing from which rod to which rod.
        For example, move (0,2) is place the top disk of rod 0 onto rod 2.
        Also, the attribute towers should contain the end solution where all disks are 
        in the correct order on rod 2.

        This class instance should at least contain the following attributes after being called:
            :param tower: A tuple with the tree rods each rod is represented by a list 
                          with integers where the integer is the size of the disk.
            :type tower: tuple[list[int]]
            :param moves: A list with all the moves to solve the problem
            :typem moves: list[tuple[int]]
            :param history: A set containing all states that are already visited.
            :type history: set[tuple[tuple[int]]]

        :param n: This is the height of the tower, i.e., the number of disks.
        :type n: int
        :return: A list containing all the moves to solve the tower of Hanoi with n disks.
        :rtype: list[tuple[int]]
        """
        self.moves = []  # a list to store the moves
        self.towers = (list(range(n, 0, -1)), [], [])
        self.history = {self.to_hashable_state(self.towers)}
        self.next_step()  # we first need the know which the next actions can be before we can take a step
        return self.moves

    @staticmethod
    def to_hashable_state(state):
        """
        This method makes a state into a hashable object.
        
        As we have seen last week sets can quickly find objects.
        However, to do this these objects must be hashable.
        A simple rule to know if a type is hashable is to ask if it is immutable,
        if not then often it is also not hashable because if you change the value
        the output of the hash function would also change.

        :param state: A current state of the tower of Hanoi problem, i.e., the current rod disk configuration.
        :type state: tuple[list[int]]
        :return: A state space that is hashable. In this case, also immutable.
        :rtype: tuple[tuple[int]]
        """
        raise NotImplementedError("Please complete this method")        

    def step(self, actions):
        """
        One step in the recursive depth-first search algorithm.

        Hint1: Do not forget to check if state has already been visited and 
               update the history as needed.
        Hint2: You only need to find one path, so as long as the path is correct
               you do not to explore any more possible actions.

        :param actions: A set of correct actions that can taken from this state.
        :type actions: list[tuple[int]]
        :return: If the current step is correct or not
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")
    
    def next_step(self):
        """
        This method helps us determine the next set of correct actions.
        This set of correct actions should be a subset of the class attribute `possible_actions`.

        :return: If the current step is correct or not
        :rtype: boolean
        """
        raise NotImplementedError("Please complete this method")

    def do_move(self, action):
        """
        This is a helper method that does one move.
        One move consists of changing one disk from one rod to another and
        to save it the move.
        
        :param action: A correct action that is taken from this state.
        :type action: tuple[int]
        """
        raise NotImplementedError("Please complete this method")

    def clean_up(self, action):
        """
        Clean up the previous move, if the current action taken does not lead to a correct solution.
        For example, you got stuck because there are no moves that go to a state that is not visited.

        :param action: A correct action that is taken from this state.
        :type action: tuple[int]
        """
        raise NotImplementedError("Please complete this method")

## Test your code

Below, you can test your algorithm. Make sure that you test various numbers of disks and that your algorithm works for all of them. To give you a head start, the tower of Hanoi with 3 disks is provided.

In [ ]:
moves = TowerOfHanoiDepth()(3)
moves, len(moves)

## 4.3 Tower of Hanoi Breadth-First Search

Here, we will implement a breadth-first search algorithm for the tower of Hanoi. Similar to the Manhattan problem, we need to store not only the visited states but also the path that was taken. However, this time it is a little bit easier as we are only interested in one solution. Therefore each history only contains one path. Before, with the depth-first search algorithm, we saw that no possible order of action could lead to the optimal path. Before you run the algorithm think about whether breadth-first search always, sometimes, or never finds the optimal solution. In the image below, we indicated the first three layers of the depth-first search algorithm, to help you understand how breadth-first search works on this state-space.

<img src="hanoi_breadth.png" alt="drawing" width="800"/>

***Before, you start programming think about a pseudo algorithm that could traverse the state-space of a tower of Hanoi with 3 disks as shown above.***

In [ ]:
%%execwritefile exercise4_{student}_notebook.py 48 -a -s

class TowerOfHanoiBreadth():   
    # All possible actions for any tower of Hanoi State-space
    possible_actions = [(0, 1), 
                        (0, 2), 
                        (1, 2), 
                        (2, 1), 
                        (1, 0), 
                        (2, 0)] 

    def __call__(self, n):      
        """
        This method uses breadth-first search to find a solution to solve the tower of Hanoi.
        The output should be a list containing all the moves to solve the problem.
        A move is a tuple with two integers containing from which rod to which rod.
        For example, move (0,2) is place the top disk of rod 0 onto rod 2.
        Also, the attribute towers should contain the end solution where all disks are 
        in the correct order on rod 2.

        This class instance should at least contain the following attributes after being called:
            :param moves: A list with all the moves to solve the problem
            :typem moves: list[tuple[int]]
            :param history: A dictionary containing all states that are already visited as keys 
                            and with the values the moves to get there.
            :type history: dict[tuple[tuple[int]], list[tuple[int]]]

        :param n: This is the height of the tower, i.e., the number of disks.
        :type n: int
        :return: A list containing all the moves to solve the tower of Hanoi with n disks.
        :rtype: list[tuple[int]]
        """
        towers = (list(range(n, 0, -1)), [], [])
        self.queue = deque([copy.deepcopy(towers)])  # propper datastructure, however, a list would work as well
        self.history = {self.to_hashable_state(towers): []}
        
        raise NotImplementedError("Please complete this method")

    def main_loop(self):
        """
        This method contains the logic of the breadth-first search for the towers of Hanoi problem.

        It does not have any inputs nor outputs. 
        Hint, use object attributes to store results.
        """
        raise NotImplementedError("Please complete this method")

    @staticmethod
    def to_hashable_state(towers):
        """
        This method makes a state into a hashable object.
        
        As we have seen last week sets can quickly find objects.
        However, to do this these objects must be hashable.
        A simple rule to know if a type is hashable is to ask if it is immutable,
        if not then often it is also not hashable because if you change the value
        the output of the hash function would also change.

        :param towers: A current state of the tower of Hanoi problem, i.e., the current rod disk configuration.
        :type towers: tuple[list[int]]
        :return: A state space that is hashable. In this case, also immutable.
        :rtype: tuple[tuple[int]]
        """
        raise NotImplementedError("Please complete this method")
    
    def base_case(self, towers):
        """
        This method checks if the current state is the final state, where
        all disks are on the last rod in the correct order.

        :param tower: A tuple with the tree rods each rod is represented by a list 
                      with integers where the integer is the size of the disk.
        :type tower: tuple[list[int]]
        """
        raise NotImplementedError("Please complete this method")

    def step(self, towers, action):
        """
        One breadth-first search step.
        Here, you add new states to the queue, and update the history.

        Hint: To create a new state, you need to make a copy of the current towers and
              then adjust them otherwise the towers for all states are adjusted.
        
        :param tower: A tuple with the tree rods each rod is represented by a list 
                      with integers where the integer is the size of the disk.
        :type tower: tuple[list[int]]
        :param action: A correct action that is taken from this state.
        :type action: tuple[int]
        """
        raise NotImplementedError("Please complete this method")

    def next_step(self, towers):
        """
        This method helps us determine the next set of correct actions.
        This set of correct actions should be a subset of the class attribute `possible_actions`.
        
        :param tower: A tuple with the tree rods each rod is represented by a list 
                      with integers where the integer is the size of the disk.
        :type tower: tuple[list[int]]
        :return: The list of possible next actions
        :rtype: list[tuple[int]
        """
        raise NotImplementedError("Please complete this method")

## Test your code

Below, you can test your algorithm. Make sure that you test various numbers of disks and that your algorithm works for all of them. To give you a head start, the tower of Hanoi with 3 disks is provided.

In [ ]:
moves = TowerOfHanoiBreadth()(3)
moves, len(moves)

# 5.0 UNITTESTS

During this assignment, we copied all your code to the following **.py** file **"exercise4_{student}_notebook.py"**. You also tested your code along the way. However, it is possible that there are still a few errors. Therefore, it is good to run some unittest when you complete all coding. This gives you an extra chance to spot mistakes. Here, we added some unittest for you to use. Note, that they are merely a check to see if your **.py** is correct.

From this point onwards we strongly advise renaming the **"exercise4_{student}_notebook.py"** file to the correct file name that you need to hand in **"exercise4_{student}.py"**. Now, you can adjust the **"exercise4_{student}.py"** file without the risk of overwriting it when you run the notebook again. This also enables the possibility to run the unittests. Note, that from now on if you make a change in the Python file and you want to go back to the notebook later that you also make this change in the notebook. To run the unittests go to the **"unit_test.py"** file and run the file in either PyCharm, VSCode, or a terminal. You can run it in a terminal using the following command: `python -m unittest --verbose unit_test.py`. `--verbose` is optional but gives you more details about which tests fail and which succeed.

You are allowed to add your own unittests. 

## Uploading to Brightspace for Bonus

Next, you can upload your Python file with the correct name on brightspace in the bonus assignment. Follow the instructions on this brightspace page carefully to have a successful submission. After you get the feedback for this exercise you can either continue working in the Python file to fix possible bugs or you can go back to the notebook and remake the Python file. ***Please be careful, do not update your code in both the Python file and notebook at the same time!***. If you go back to the notebook do not forget to update the notebook with any changes you made within the Python file. In this case, it is best to just delete the Python file as soon as you copied all changes.

***NOTE, that you can now also upload the exercises from week 1! The process is exactly the same only there is no unittest.***